In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import time
import re
%matplotlib inline
pd.set_option("display.max_rows",25)
pd.set_option("display.max_columns",15)
pd.options.display.float_format = "{:.2f}".format

**DATA SCRAPPING**  

In [ ]:
# Generating the listing urls on yearly basis
year=['2010','2011','2012','2013','2014','2015',"2016","2017"]

pagerange=np.arange(1,11)
url=[]
dict_years_list={}


def page_range_url(year):
    for i,y in enumerate(year):
        url.append("http://www.imdb.com/search/title?year="+y+"&title_type=feature&")
        dict_years_list[y]=[]
    print(url)    
    for d, u in zip(dict_years_list.keys(),url):
        #for u in url:
        
            for itr in pagerange:
                dict_years_list[d].append(u+"&page="+str(itr)+"&ref_=adv_nxt")
    return dict_years_list


In [ ]:
#Scrapping the list search of urls year wise.
def scrap_url(dict_years_list):
    
    page=dict(zip(dict_years_list.keys(), [None]*len(dict_years_list.keys())))   
    for idx, value in (dict_years_list.items()):
        li=[]
        
        for i in value:
            print(i)
            li.append(requests.get(i).text)
            
        print(idx)   
        page[idx]=li 
    return page

In [ ]:
dict_year=page_range_url(year) # Generating the URL list 

In [ ]:
all_title_links=scrap_url(dict_year) 

In [ ]:
#pickling the list of all URLS
pd.to_pickle(all_title_links,"all_title_links.pkl") 
links=pd.read_pickle("all_title_links.pkl")

In [ ]:
links.keys()
list_2014=[]
list_2015=[]
list_2016=[]
list_2017=[]

In [ ]:
# Scrapping the list YEAR wise
def scrap_listing_url(listing):
    listof=[]
    for i,title in enumerate(listing):
        listof.append(BeautifulSoup(title,'lxml'))
    
    return listof

In [ ]:
list_2015=scrap_listing_url(links['2015'])
list_2016=scrap_listing_url(links["2016"])
list_2017=scrap_listing_url(links["2017"])
list_2014=scrap_listing_url(links["2014"])

In [ ]:
#Extracting list title links

final_list_link=[]
#final_list_name=[]
def title_part_link(listyear):
    
    for t in listyear:
        movie_links=[i.find('a')["href"] for i in t.find(class_="article").find_all(class_="lister-item-header")]
        final_list_link.append(movie_links)
        #movie_name=[a.find('a').text for a in t.find(class_="article").find_all(class_="lister-item-header")]
        #final_list_name.append(movie_name)
    return (final_list_link)

In [ ]:
#Flattening the list of list
flatten = lambda l: [item for sublist in l for item in sublist]
final_list_links=flatten(title_part_link(list_2016))

In [ ]:
linker_2017=[]
linker_2015=[]
linker_2016=[]
linker_2014=[]
for f in final_list_links:
    linker_2017.append("http://www.imdb.com/"+f)
    

In [ ]:
#pickling and retrieving 

pd.to_pickle(linker_2015,"linker_2015.pkl")
pd.to_pickle(linker_2016,"linker_2016.pkl")
pd.to_pickle(linker_2017,"linker_2017.pkl")
pd.to_pickle(linker_2014,"linker_2014.pkl")

linker_2014=pd.read_pickle("linker_2014.pkl")
linker_2015=pd.read_pickle("linker_2015.pkl")
linker_2016=pd.read_pickle("linker_2016.pkl")
linker_2017=pd.read_pickle("linker_2017.pkl")

In [ ]:
# Scrapping the Final Title Page
def scrap_title(linker):
    listy=[]
    for indx,l in enumerate (linker):
        print(indx)
        print(l)
        listy.append(requests.get(l).text)
    return listy

In [ ]:
jumbo_2014_1=scrap_title(linker_2014)
jumbo_2015_1=scrap_title(linker_2015)
jumbo_2016_1=scrap_title(linker_2016)
jumbo_2017_1=scrap_title(linker_2017)


In [ ]:
#final content scrapper of the title of the movie
def scrap_details(list_name):
    import time
    import re
    budget=[]
    cast=[]
    plot=[]
    budget=[]
    gross_usa=[]
    cumm_world=[]
    director=[]
    genre=[]
    language=[]
    country=[]
    release_date=[]
    num_reviews=[]
    run_time=[]
    content_rating=[]
    rating=[]
    name_movie=[]
    for i,title in enumerate(list_name):
        beauty=(BeautifulSoup(title,'lxml'))
        casty=[]
        print(i)
#Movie Name    
        name_movie.append(beauty.find("h1",itemprop="name").text.replace(u"\xa0",""))
#Cast    
        if(beauty.find_all(class_="itemprop")!=None):
            for element in beauty.find_all(class_="itemprop"):
                if(element.find('a') !=None):
                    casty.append(element.find('a').text.replace("\n","").strip())
        else:
            print("Yes")
            casty.append(None)    
        cast.append(casty)        
#Plot    
        try:
        
            plot.append(beauty.find(class_="inline canwrap",itemprop="description").text)
                
        except:
            print("No Plot")
            plot.append(None)
#Genre            
        for element in beauty.find_all(itemprop="genre"):
            c=element.find_all("a")
            t=[]
            for itr in c:
                if itr!=None:
                    t.append(itr.text.strip())
                else:
                    t.append(None)
        genre.append(t)      
#Director    
        try:
        #if(beauty.find(class_="credit_summary_item").find('a').text!=None):
            director.append(beauty.find(class_="credit_summary_item").find('a').text)
        except:
            director.append(None)
#Language
        if(beauty.find('h4', text="Language:")!=None):
            language.append(beauty.find('h4', text="Language:").find_next_sibling().text)
        else:
            language.append(None)
    
#Country
        if(beauty.find('h4', text="Country:")!=None):
            country.append(beauty.find('h4', text="Country:").find_next_sibling().text)
        else:
            country.append(None)
        
#Release Date
        if(beauty.find("meta", itemprop="datePublished")!=None):
            release_date.append(beauty.find("meta", itemprop="datePublished")["content"])
        else:
            release_date.append(None)
        
#Movie Run Time
        if(beauty.find("h4", text="Runtime:")!=None):
            run_time.append(beauty.find("h4", text="Runtime:").find_next_sibling().text.split(" ")[0])
        else:
            run_time.append(None)
        
#Number of Revies
        if(beauty.find(class_="ratingValue")!= None):
               num_reviews.append(beauty.find(class_="ratingValue").find_next_sibling().text)
        else:
               num_reviews.append(None)
    
#Content Rating
        if(beauty.find("meta",itemprop="contentRating")!=None):
            content_rating.append(beauty.find("meta",itemprop="contentRating")["content"])
        else:
            content_rating.append(None)
#Rating (Censor)
        if (beauty.find("span",itemprop="ratingValue")!=None):
            rating.append(beauty.find("span",itemprop="ratingValue").text)
        else:
            rating.append(None)
#Movie Budget    
        if(beauty.find("h4",text="Budget:")!=None):
            budget.append(beauty.find("h4",text="Budget:").next_sibling.replace("\n","").strip().replace("$",""))
        else:
            budget.append(None)
#Gross Collection USA
        if(beauty.find("h4",text="Gross USA:")!=None):
        
            gross_usa.append(beauty.find("h4",text="Gross USA:").next_sibling.replace("\n","").strip().replace("$",""))
        else:    
            gross_usa.append(None)
#Gross Collections Worldwide
        if(beauty.find("h4",text="Cumulative Worldwide Gross:")!=None):
            cumm_world.append(beauty.find("h4",text="Cumulative Worldwide Gross:").next_sibling.replace("\n","").strip().replace("$",""))
        else:
            cumm_world.append(None)
    return name_movie,plot,cast,budget,rating,content_rating,gross_usa,cumm_world,num_reviews,run_time,language,director,genre,release_date,country

In [ ]:
MOVIE,PLOT,CAST,BUDGET,RATING,CONTENT_RATING,GROSS_REV,CUMM_REV,REVIEWS,RUN_TIME,LANGUAGE,DIRECTOR,GENRE,RELEASE_DATE,COUNTRY=scrap_details(jumbo_2016_1)

In [ ]:
# df_2016=pd.DataFrame({"MOVIE":MOVIE,"CAST":CAST
#               ,"BUDGET":BUDGET,"RATING":RATING,"CONTENT_RATING":CONTENT_RATING,
#               "GROSS_REV":GROSS_REV,"CUMM_REV":CUMM_REV,"REVIEWS":REVIEWS,
#               "RUN_TIME":RUN_TIME,"LANGUAGE":LANGUAGE,"DIRECTOR":DIRECTOR,"GENRE":GENRE,"RELEASE_DATE":RELEASE_DATE,"PLOT":PLOT,"COUNTRY":COUNTRY})

In [ ]:
#pd.to_pickle(df_2016,"df_2701_2016.pkl")